### MarineTraffic Power Tool - Fleets (Modify Fleet)


API documentation https://servicedocs.marinetraffic.com/tag/Fleets

API key can be found at https://www.marinetraffic.com/en/users/my_account/api/account



In [17]:
import time
import requests  # type: ignore
import xml.etree.ElementTree as ET
import pandas as pd # type: ignore

In [ ]:
# Variables to adjust
api_key_getfleet = "apikey1"  # Get vessels in fleet
api_key_setfleet = "apikey2"  # Update/Add fleet URL
api_key_clearfleet = "apikey3"  # Clear fleet URL
api_key_listfleetids = "apikey4"  # Retrieve fleet IDs

# Editable variable for the file path
file_path = "E:/example-vessel-list.csv"  # Change this to your desired path

# Max vessels allowed in a fleet
MAX_VESSELS_PER_FLEET = 2500

# Fleet IDs where vessels will be inserted
fleet_ids = ["2558323", "2558324", "2558325"]  # Example of multiple fleets

identifier = "imo"  # Can be "imo", "mmsi", or "ship_id"
# Example IMO numbers (or MMSI, or SHIP_ID depending on identifier)
VESSELS = []

# API URL templates
set_fleet_url_template = f"https://services.marinetraffic.com/api/setfleet/{api_key_setfleet}/{identifier}"  # Update/Add fleet URL
clear_fleet_url_template = f"https://services.marinetraffic.com/api/clearfleet"  # Clear fleet URL
list_fleets_url = f"https://services.marinetraffic.com/api/getfleets/{api_key_listfleetids}/protocol:xml"  # List fleet IDs URL

In [19]:
# Step 1: Fetch current vessels in a specific fleet
def fetch_vessels_in_fleet(fleet_id):
    get_fleet_url = f"https://services.marinetraffic.com/api/getfleet/{api_key_getfleet}/v:2/fleet_id:{fleet_id}/protocol:xml"
    response = requests.get(get_fleet_url)

    if response.status_code == 200:
        try:
            # Parse the XML response
            root = ET.fromstring(response.content)
            vessels_in_fleet = []

            # Loop through the VESSEL elements
            for vessel in root.findall('VESSEL'):
                vessel_data = {
                    "ship_id": vessel.get("SHIP_ID"),
                    "mmsi": vessel.get("MMSI"),
                    "imo": vessel.get("IMO"),
                    "shipname": vessel.get("SHIPNAME"),
                    "active": vessel.get("ACTIVE"),
                }
                vessels_in_fleet.append(vessel_data)

            return vessels_in_fleet

        except ET.ParseError as e:
            print(f"Failed to parse XML. Error: {e}.")
            return []
    else:
        print(f"Error fetching vessels: {response.status_code}.")
        return []

# Step 2: Identify vessels to add
def identify_vessels_to_add(vessels_in_fleet, vessels_to_add):
    # Get a set of identifiers already in the fleet based on the chosen identifier variable
    fleet_identifiers = {vessel[identifier] for vessel in vessels_in_fleet}  # Use the dynamic identifier

    # Identify vessels to add that are not in the fleet
    to_add = [vessel for vessel in vessels_to_add if vessel not in fleet_identifiers]

    return to_add

# Step 3: Add vessels to multiple fleets, respecting the 2.5k limit
def add_vessels_to_fleets(vessels_to_add):
    current_fleet_idx = 0

    while vessels_to_add:
        # Get the current fleet ID
        if current_fleet_idx >= len(fleet_ids):
            # Not enough fleet IDs to accommodate all vessels
            print(f"Error: Not enough fleet IDs provided to add all vessels. {len(vessels_to_add)} vessels remain.")
            print("Please provide more fleet IDs and try again.")
            return

        current_fleet_id = fleet_ids[current_fleet_idx]

        # Fetch vessels in the current fleet
        vessels_in_fleet = fetch_vessels_in_fleet(current_fleet_id)

        # If the current fleet has space, add vessels to it
        available_space = MAX_VESSELS_PER_FLEET - len(vessels_in_fleet)
        if available_space > 0:
            # Get the vessels to add to this fleet
            vessels_for_this_fleet = vessels_to_add[:available_space]
            vessels_to_add = vessels_to_add[available_space:]  # Update the remaining vessels

            # Identify vessels that need to be added
            vessels_to_add_for_fleet = identify_vessels_to_add(vessels_in_fleet, vessels_for_this_fleet)

            print(f"Adding {len(vessels_to_add_for_fleet)} vessels to fleet {current_fleet_id}...")

            # Add the vessels to the fleet
            failed_vessels = []
            for vessel_number in vessels_to_add_for_fleet:
                set_fleet_url = f"{set_fleet_url_template}:{vessel_number}/fleet_id:{current_fleet_id}/active:1"
                response = requests.get(set_fleet_url)

                if response.status_code == 200:
                    print(f"Vessel {vessel_number} added to fleet {current_fleet_id}.")
                else:
                    print(f"Error adding vessel {vessel_number}: {response.text}")
                    failed_vessels.append(vessel_number)

                time.sleep(5)  # Buffer to avoid overwhelming the API

            if failed_vessels:
                print(f"The following vessels were not added due to errors: {failed_vessels}")
        else:
            print(f"Fleet {current_fleet_id} is full. Moving to the next fleet.")

        # Move to the next fleet
        current_fleet_idx += 1

# Step 4: Load vessels from file or use the VESSELS array
def load_vessels_from_file():
    # Load vessels from CSV file
    try:
        df = pd.read_csv(file_path)
        vessels = df[identifier].tolist()  # Use the identifier to get the correct column
        return vessels
    except Exception as e:
        print(f"Error reading vessels from file: {e}")
        return []

# Step 5: Remove vessels from the fleet for multiple fleet_ids
def remove_vessels_from_multiple_fleets(vessels_to_remove, fleet_ids):
    for fleet_id in fleet_ids:
        vessels_in_fleet = fetch_vessels_in_fleet(fleet_id)  # Fetch vessels for the current fleet
        fleet_identifiers = {vessel[identifier]: vessel for vessel in vessels_in_fleet}

        failed_vessels = []

        for vessel_number in vessels_to_remove:
            if vessel_number in fleet_identifiers:
                # Format the set fleet URL with the correct vessel number and delete parameter
                set_fleet_url = f"{set_fleet_url_template}:{vessel_number}/fleet_id:{fleet_id}/delete:1"
                response = requests.get(set_fleet_url)

                if response.status_code == 200:
                    print(f"Vessel {vessel_number} removed successfully from fleet {fleet_id}.")
                else:
                    print(f"Error removing vessel {vessel_number} from fleet {fleet_id}: {response.text}")
                    failed_vessels.append(vessel_number)
            else:
                print(f"Vessel {vessel_number} is not in the fleet {fleet_id}.")
                failed_vessels.append(vessel_number)

            time.sleep(5)

        if failed_vessels:
            print(f"The following vessels could not be removed from fleet {fleet_id}: {failed_vessels}")

# Step 6: Set vessels active or inactive for multiple fleet_ids
def set_vessel_active_for_multiple_fleets(vessels_to_set_active, fleet_ids):
    for fleet_id in fleet_ids:
        failed_vessels = []

        for vessel_number in vessels_to_set_active:
            set_fleet_url = f"{set_fleet_url_template}:{vessel_number}/fleet_id:{fleet_id}/active:1"
            response = requests.get(set_fleet_url)

            if response.status_code == 200:
                print(f"Vessel {vessel_number} set to active in fleet {fleet_id} successfully.")
            else:
                print(f"Error setting vessel {vessel_number} to active in fleet {fleet_id}: {response.text}")
                failed_vessels.append(vessel_number)

            time.sleep(5)

        if failed_vessels:
            print(f"The following vessels could not be set to active in fleet {fleet_id}: {failed_vessels}")

def set_vessel_inactive_for_multiple_fleets(vessels_to_set_inactive, fleet_ids):
    for fleet_id in fleet_ids:
        failed_vessels = []

        for vessel_number in vessels_to_set_inactive:
            set_fleet_url = f"{set_fleet_url_template}:{vessel_number}/fleet_id:{fleet_id}/active:0"
            response = requests.get(set_fleet_url)

            if response.status_code == 200:
                print(f"Vessel {vessel_number} set to inactive in fleet {fleet_id} successfully.")
            else:
                print(f"Error setting vessel {vessel_number} to inactive in fleet {fleet_id}: {response.text}")
                failed_vessels.append(vessel_number)

            time.sleep(5)

        if failed_vessels:
            print(f"The following vessels could not be set to inactive in fleet {fleet_id}: {failed_vessels}")


# Step 7: List vessels in a specific fleet using XML protocol
def list_vessels_in_fleet(fleet_id):
    # Update the fleet URL to include the selected fleet ID
    get_fleet_url = f"https://services.marinetraffic.com/api/getfleet/{api_key_getfleet}/v:2/fleet_id:{fleet_id}/protocol:xml"

    try:
        response = requests.get(get_fleet_url)

        if response.status_code == 200:
            # Parse the XML response
            root = ET.fromstring(response.content)
            vessels_in_fleet = []

            # Loop through the VESSEL elements
            for vessel in root.findall('VESSEL'):
                vessel_data = {
                    "ship_id": vessel.get("SHIP_ID"),
                    "mmsi": vessel.get("MMSI"),
                    "imo": vessel.get("IMO"),
                    "shipname": vessel.get("SHIPNAME"),
                    "active": vessel.get("ACTIVE"),
                }
                vessels_in_fleet.append(vessel_data)

            if vessels_in_fleet:
                return vessels_in_fleet
            else:
                print(f"No vessels found in fleet ID: {fleet_id}.")
                return []
        else:
            print(f"Error fetching vessels for fleet {fleet_id}: {response.status_code}.")
            return []

    except Exception as e:
        print(f"An error occurred while fetching vessels for fleet {fleet_id}: {e}")
        return []

# Step 8: Clear vessels from multiple fleets
def clear_vessels_in_multiple_fleets(fleet_ids):
    for fleet_id in fleet_ids:
        vessels_in_fleet = fetch_vessels_in_fleet(fleet_id)

        if not vessels_in_fleet:
            print(f"Fleet {fleet_id} is already empty.")
            continue

        confirmation = input(f"Are you sure you want to remove all vessels in fleet {fleet_id}? (Y/N): ").strip().upper()

        if confirmation == 'Y':
            clear_fleet_url = f"{clear_fleet_url_template}/{api_key_clearfleet}/fleet_id:{fleet_id}"
            response = requests.get(clear_fleet_url)

            if response.status_code == 200:
                print(f"All vessels in fleet {fleet_id} have been cleared successfully.")
            else:
                print(f"Error clearing fleet {fleet_id}: {response.status_code}. Response: {response.text}")
        elif confirmation == 'N':
            print(f"Fleet clearance operation for fleet {fleet_id} canceled.")
        else:
            print("Invalid option. Please enter Y or N.")

# Step 9: List fleet ids using XML protocol and handle multiple fleet_ids
def list_fleet_ids():
    get_fleet_url = f"https://services.marinetraffic.com/api/getfleets/{api_key_listfleetids}/protocol:xml" 
    response = requests.get(get_fleet_url)

    if response.status_code == 200:
        try:
            # Parse the XML response
            root = ET.fromstring(response.content)

            # Find all FLEET elements
            fleets = root.findall('FLEET')

            fleet_list = []

            if fleets:
                print("Fleets found:")
                for fleet in fleets:
                    fleet_info = {
                        "ID": fleet.get("ID", "N/A"),
                        "NAME": fleet.get("NAME", "N/A"),
                        "ACTIVE": fleet.get("ACTIVE", "N/A"),
                        "CREATED": fleet.get("CREATED", "N/A"),
                        "DEFAULT": fleet.get("DEFAULT", "N/A")
                    }
                    print(f"Fleet ID: {fleet_info['ID']}, Name: {fleet_info['NAME']}, Active: {fleet_info['ACTIVE']}")
                    fleet_list.append(fleet_info)
            else:
                print("No fleets found.")

            return fleet_list

        except ET.ParseError:
            print("Failed to parse XML. Response content is not in valid XML format.")
    else:
        print(f"Error fetching fleets: {response.status_code}.")
        return []

In [ ]:
# Main script to prompt user for action
def main():
    fleets = list_fleet_ids()  # Fetch all fleet IDs at the start
    if not fleets:
        print("No fleets available. Exiting.")
        return

    # Allow the user to select specific fleet IDs or use all fleets
    print("\nAvailable fleets:")
    for idx, fleet in enumerate(fleets, start=1):
        print(f"{idx}. Fleet ID: {fleet['ID']}, Name: {fleet['NAME']}")
    
    selected_fleet_input = input("Enter the number(s) of the fleet(s) you want to work with (comma-separated) or press Enter to use all fleets: ")
    if selected_fleet_input:
        selected_fleet_indexes = [int(i) - 1 for i in selected_fleet_input.split(',')]
        fleet_ids = [fleets[i]['ID'] for i in selected_fleet_indexes]
    else:
        fleet_ids = [fleet['ID'] for fleet in fleets]

    while True:
        print("\nChoose an action:")
        print("1. Add vessels")
        print("2. Remove vessels")
        print("3. Set vessels active")
        print("4. Set vessels inactive")
        print("5. List vessels in fleet")
        print("6. Clear all vessels in fleet")
        print("7. List Fleet IDs")
        print("8. Exit")

        choice = input("Enter your choice (1-8): ")

        if choice == "1":
            vessels_to_add = VESSELS if VESSELS else load_vessels_from_file()
            add_vessels_to_fleets(vessels_to_add, fleet_ids)
        elif choice == "2":
            vessels_to_remove = VESSELS if VESSELS else input("Enter vessel identifiers to remove (comma-separated): ").split(',')
            remove_vessels_from_multiple_fleets(vessels_to_remove, fleet_ids)
        elif choice == "3":
            vessels_to_set_active = VESSELS if VESSELS else input("Enter vessel identifiers to set active (comma-separated): ").split(',')
            set_vessel_active_for_multiple_fleets(vessels_to_set_active, fleet_ids)
        elif choice == "4":
            vessels_to_set_inactive = VESSELS if VESSELS else input("Enter vessel identifiers to set inactive (comma-separated): ").split(',')
            set_vessel_inactive_for_multiple_fleets(vessels_to_set_inactive, fleet_ids)
        elif choice == "5":
            for fleet_id in fleet_ids:
                print(f"\nListing vessels for fleet ID: {fleet_id}")
                vessels_in_fleet = list_vessels_in_fleet(fleet_id)
                if vessels_in_fleet:
                    for vessel in vessels_in_fleet:
                        print(vessel)
                else:
                    print(f"No vessels found in fleet {fleet_id}.")
        elif choice == '6':
            clear_vessels_in_multiple_fleets(fleet_ids)
        elif choice == '7':
            fleets = list_fleet_ids()  # Refresh fleet list
            if not fleets:
                print("No fleets found.")
            else:
                for idx, fleet in enumerate(fleets, start=1):
                    print(f"{idx}. Fleet ID: {fleet['ID']}, Name: {fleet['NAME']}")
        elif choice == "8":
            print("Exiting the program.")
            break
        else:
            print("Invalid choice. Please enter a number between 1 and 8.")

if __name__ == "__main__":
    main()
